# Info bici

In [2]:
import json
import pickle
from shapely import MultiLineString, LineString, Point

import geopandas as gpd
import osmnx as ox
import networkx as nx

from my_paths import *
import Graph_functions

## Strade/Ciclabili

### Piste ciclabili

Prima fase di pulizia del geojson raw

In [3]:
gdf_ciclabili_raw = gpd.read_file(PATH_CICLABILI_RAW)
gdf_ciclabili_staging = gdf_ciclabili_raw.copy()

# Convertiamo alcuni tipi
gdf_ciclabili_staging["geometry"] = gdf_ciclabili_staging["geometry"].apply(lambda x: x.geoms[0])
gdf_ciclabili_staging["fine_lavor"] = gdf_ciclabili_staging["fine_lavor"].astype(int)
gdf_ciclabili_staging = gdf_ciclabili_staging.rename(columns={"anagrafica": "name", "rete": "highway"})
gdf_ciclabili_staging = gdf_ciclabili_staging.drop(["id_amat", "id_via","gerarchia", "lunghezza"], axis=1)

gdf_ciclabili_staging = gpd.GeoDataFrame(gdf_ciclabili_staging, geometry="geometry")

Creazione del grafo + archi di congiunzione tra nodi molto vicini (5 metri).
Aggiunta di alcuni attributi utili:
- **"tipo"** = Per identificare in futuro la provenienza degli archi.
- **"artificiale"** = "False" se riferiti a veri percorsi presi dal geojson, "True" se aggiunti per scopi di calcolo percorsi.
N.B: "geojson_to_graph()" manterrà all'interno degli archi del grafo anche tutti gli attributi delle colonne del gdf.

In [4]:
G_ciclabili = Graph_functions.geojson_to_graph(gdf_ciclabili_staging, weight_moltiplicator=0, tipo = "Ciclabile", artificiale = False).to_undirected()
G_ciclabili = Graph_functions.add_edge_near_nodes(G_ciclabili, weight_moltiplicator = 0, tipo = "Ciclabile", artificiale = True)

Salviamo grafo (in pickle) e geojson in staging

In [5]:
gdf_edges = ox.graph_to_gdfs(G_ciclabili, edges=True, nodes=False).reset_index().drop(["u","v", "key"], axis=1)
gdf_edges.to_file(PATH_CICLABILI_GEOJSON_STAGING, driver="GeoJSON") # Anche questo andrebbe ripulito
with open(PATH_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_ciclabili, f)

### Strade ciclabili

#### Ingestion (openstreet maps)

In [ ]:
place = "Milano, Italy"

bike_filter = (
    '["highway"]["bicycle"!~"no"]["bicycle"!~"dismount"]'
)
G_ox_strade = ox.graph_from_place(place, network_type='bike', custom_filter=bike_filter)
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "wb") as f:
    pickle.dump(G_ox_strade, f)

#### ETL 

1. Scelta attributi da mantenere
2. Impostare pesi (weight) in base a: Lunghezza strada e tipo di strada
3. Aggiunti di attributi custom, come:
    - **"artificiale"** = Di base "False" ma sarà "True" con aggiunta di archi ad hoc a scopi di calcolo percorsi minimi
    - **"geometry"** = Manteniamo il dato originale della geometria all'interno degli archi. Utile per conversione in gdf.
    - **"tipo"** = Attributo per distinguere la provenienza degli archi. Utile quando verranno uniti in un solo grafo.
4. Aggiunta di archi artificiali per connettere nodi molto vicini non connessi.

In [6]:
def calcola_peso_strada(highway) -> float:
    custom_weights = {
        "motorway": 99,
        "trunk": 99,
        "primary": 1.2,
        "secondary": 1.1,
        "tertiary": 1.25,
        "residential": 1.3,
        "unclassified": 1.3,
        "living_street": 1.3,
        "motorway_link": 99,
        "trunk_link": 99,
        "primary_link": 1.2,
        "secondary_link": 1.1,
        "tertiary_link": 1.25,
        "service": 99,
        "footway": 1.2,
        "pedestrian": 1.5,
        "steps": 99,
        "platform": 99,
        "path": 1.1,
        "cycleway": 1,
        "track": 1,
        "bridleway": 1.6,
        "construction": 99,
        "proposed": 99,
        "demolished": 99,
        "busway": 99,
        "bus_stop": 99,
        "corridor": 99,
        "services": 99,
        "emergency_bay": 99
    }
    if isinstance(highway, str):
        highway = [highway]
    max_weight = 99
    for h in highway:
        new_weight = custom_weights.get(h, 99)
        if new_weight < max_weight:
            max_weight = new_weight
    return max_weight

In [10]:
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "rb") as f:
    G_strade = pickle.load(f)
for u, v, k, data in G_strade.edges(data=True, keys=True):
    # Copio i data ed elimino tutto il dizionario.
    data = data.copy()
    G_strade[u][v][k].clear()
    # Scelgo gli attributi da mantenere
    G_strade[u][v][k]['length'] = data['length']
    G_strade[u][v][k]['name'] = data.get('name', None)
    G_strade[u][v][k]['highway'] = data["highway"]
    weight_multipler = calcola_peso_strada(data["highway"])
    G_strade[u][v][k]["weight_multipler"] = weight_multipler
    G_strade[u][v][k]['weight'] = data['length'] * weight_multipler # Per ora
    G_strade[u][v][k]["tipo"] = "Strade_ciclabili"
    G_strade[u][v][k]["artificiale"] = False
    # ------------ Da capire se tenere ----------------
    G_strade[u][v][k]["maxspeed"] = data.get("maxspeed", None)
    G_strade[u][v][k]["tunnel"] = data.get("tunnel", None)
    G_strade[u][v][k]["access"] = data.get("access", None)
    G_strade[u][v][k]["service"] = data.get("service", None)
    # ------------------------------------------------
    G_strade[u][v][k]["geometry"] = data.get("geometry", LineString([(G_strade.nodes[u]["x"], G_strade.nodes[u]["y"]),
                                        (G_strade.nodes[v]["x"], G_strade.nodes[v]["y"])]))
nx.set_node_attributes(G_strade, "Strade_ciclabili", "tipo")
G_strade = G_strade.to_undirected()

Salvataggio in staging, sia grafo che geojson.

In [16]:
gdf_edges = ox.graph_to_gdfs(G_strade, edges=True, nodes=False).reset_index().drop(["u", "v", "key"], axis=1)

gdf_edges.to_file(PATH_STRADE_CICLABILI_GEOJSON_STAGING, driver="GeoJSON")

with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_strade, f)

### Unione Ciclabili e Strade

Load da staging i 2 Grafi:
- Piste **Ciclabili**
- **Strade** in cui è permesso usare la bici

In [3]:
# Carichiamo Graph Ciclabili
with open(PATH_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_ciclabili = pickle.load(f)

# Carichiamo Graph Strade
with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_strade = pickle.load(f)

1. Unione dei due Grafi tramite: **nx.compose()**.
2. Aggiunta di archi artificiali per unire i nodi dei 2 Grafi, in modo che possano essere "comunicanti"  
(si possa passare da un arco "strade" a un arco "ciclabili" e viceversa)

In [5]:
G_compose = Graph_functions.connetti_due_grafi(G_strade, G_ciclabili, weight_moltiplicator=1,
                                                tipo = "Join_ciclabile_strade",
                                                artificiale=True)

In [6]:
with open(PATH_RETE_CICLABILE_COMPLETA_PICKLE_CLEAN, "wb") as f:
    pickle.dump(G_compose, f)

gdf_compose = ox.graph_to_gdfs(G_compose, edges=True, nodes=False).reset_index().drop(["u","v", "key"], axis=1)
gdf_compose.to_file(PATH_RETE_CICLABILE_COMPLETA_GEOJSON_CLEAN, driver="GeoJSON")